# Fixed Format Ensemble - Exclude ALL Bad Snapshots

exp_005 has 201 rows with missing 's' prefix.
Bad snapshots to exclude:
- 21145963314 (wrong ID + no 's')
- 21337107511 (wrong ID + no 's')
- 21145965159 (no 's' prefix)
- 21336527339 (no 's' prefix)

In [ ]:
import os
import json
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union

# ALL bad snapshots to exclude
BAD_SNAPSHOTS = {'21145963314', '21337107511', '21145965159', '21336527339'}

# Tree polygon vertices for score calculation
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def create_tree_polygon(x, y, angle):
    poly = Polygon(zip(TX, TY))
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

def calculate_side(trees):
    polys = [create_tree_polygon(*t) for t in trees]
    union = unary_union(polys)
    bounds = union.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def calculate_score_for_n(trees, n):
    side = calculate_side(trees)
    return (side ** 2) / n

print(f"Bad snapshots to exclude: {BAD_SNAPSHOTS}")

In [ ]:
def load_snapshot_with_full_validation(path):
    """Load snapshot with BOTH ID format AND 's' prefix validation."""
    rows_by_n = {}
    with open(path, 'r') as f:
        next(f)  # Skip header
        for line in f:
            parts = line.strip().split(',')
            if len(parts) != 4:
                continue
            id_val, x, y, deg = parts
            
            # Validate ID format
            try:
                n_str, idx_str = id_val.split('_')
                n = int(n_str)
                idx = int(idx_str)
                expected_id = f"{n:03d}_{idx}"
                if id_val != expected_id:
                    return None  # Wrong ID format
            except:
                return None
            
            # Validate 's' prefix on coordinates
            if not (x.startswith('s') and y.startswith('s') and deg.startswith('s')):
                return None  # Missing 's' prefix
            
            if n not in rows_by_n:
                rows_by_n[n] = []
            rows_by_n[n].append(parts)
    return rows_by_n

def parse_row_to_tuple(row):
    """Parse a row to (x, y, angle) tuple for score calculation."""
    x_str = row[1]
    y_str = row[2]
    deg_str = row[3]
    x = float(x_str[1:] if x_str.startswith('s') else x_str)
    y = float(y_str[1:] if y_str.startswith('s') else y_str)
    angle = float(deg_str[1:] if deg_str.startswith('s') else deg_str)
    return (x, y, angle)

def calculate_score_from_rows(rows, n):
    """Calculate score from raw string rows."""
    trees = [parse_row_to_tuple(row) for row in rows]
    return calculate_score_for_n(trees, n)

print("Loading functions with full validation defined!")

In [ ]:
# Load baseline (exp_001 which passed Kaggle) as raw strings
baseline_path = '/home/nonroot/snapshots/santa-2025/21145966992/submission/submission.csv'
baseline_raw = load_snapshot_with_full_validation(baseline_path)

if baseline_raw is None:
    print("ERROR: Baseline has format issues!")
else:
    # Calculate baseline scores
    baseline_scores = {n: calculate_score_from_rows(baseline_raw[n], n) for n in range(1, 201)}
    baseline_total = sum(baseline_scores.values())
    print(f"Baseline total score: {baseline_total:.6f}")
    print(f"Baseline N=2 rows: {baseline_raw[2]}")

In [ ]:
# Initialize best_per_n with baseline
best_per_n = {
    n: {
        'score': baseline_scores[n],
        'rows': baseline_raw[n],
        'source': 'baseline'
    } for n in range(1, 201)
}

print("Initialized with baseline")

In [ ]:
# Load all snapshots and find improvements (excluding ALL bad snapshots)
snapshot_base = '/home/nonroot/snapshots/santa-2025/'
snapshot_dirs = sorted(os.listdir(snapshot_base))
print(f"Found {len(snapshot_dirs)} snapshot directories")
print(f"Excluding bad snapshots: {BAD_SNAPSHOTS}")

improvements_found = 0
snapshots_processed = 0
snapshots_skipped = 0

for snap_dir in snapshot_dirs:
    # Skip known bad snapshots
    if snap_dir in BAD_SNAPSHOTS:
        snapshots_skipped += 1
        continue
    
    sub_path = os.path.join(snapshot_base, snap_dir, 'submission', 'submission.csv')
    if not os.path.exists(sub_path):
        continue
    
    try:
        snap_raw = load_snapshot_with_full_validation(sub_path)
        
        if snap_raw is None:
            # This snapshot has format issues
            snapshots_skipped += 1
            continue
        
        # Check each N
        for n in range(1, 201):
            if n not in snap_raw:
                continue
            
            rows = snap_raw[n]
            score = calculate_score_from_rows(rows, n)
            
            # Only accept if better score
            if score < best_per_n[n]['score'] - 1e-10:
                best_per_n[n]['score'] = score
                best_per_n[n]['rows'] = rows  # Keep original string rows!
                best_per_n[n]['source'] = snap_dir
                improvements_found += 1
        
        snapshots_processed += 1
        if snapshots_processed % 20 == 0:
            print(f"Processed {snapshots_processed} snapshots, {improvements_found} improvements...")
                    
    except Exception as e:
        snapshots_skipped += 1

print(f"\nTotal: {snapshots_processed} snapshots processed")
print(f"Snapshots skipped (format issues): {snapshots_skipped}")
print(f"Improvements found: {improvements_found}")

In [ ]:
# Calculate new total score
new_total = sum(best_per_n[n]['score'] for n in range(1, 201))
print(f"New total score: {new_total:.6f}")
print(f"Baseline total: {baseline_total:.6f}")
print(f"Improvement: {baseline_total - new_total:.6f}")

# Count unique sources
sources = set(best_per_n[n]['source'] for n in range(1, 201))
print(f"\nUnique sources used: {len(sources)}")

In [ ]:
# Show top improvements
print("\nTop 20 improvements:")
improved_n = []
for n in range(1, 201):
    if best_per_n[n]['source'] != 'baseline':
        old_score = baseline_scores[n]
        new_score = best_per_n[n]['score']
        improved_n.append((n, old_score - new_score, best_per_n[n]['source']))

for n, improvement, source in sorted(improved_n, key=lambda x: -x[1])[:20]:
    print(f"  N={n}: improved by {improvement:.6f} from {source}")

print(f"\nTotal N values improved: {len(improved_n)}")

In [ ]:
# Write ensemble preserving original string precision
def write_ensemble(best_per_n, output_path):
    """Write ensemble using original string rows (preserves precision)."""
    with open(output_path, 'w') as f:
        f.write('id,x,y,deg\n')
        for n in range(1, 201):
            for row in best_per_n[n]['rows']:
                f.write(','.join(row) + '\n')

write_ensemble(best_per_n, '/home/submission/submission.csv')
print("Saved ensemble to /home/submission/submission.csv")

In [ ]:
# MANDATORY: Validate ALL format requirements before submission
def validate_submission(path):
    """Verify ALL format requirements."""
    errors = []
    with open(path, 'r') as f:
        next(f)  # Skip header
        for line_num, line in enumerate(f, 2):
            parts = line.strip().split(',')
            if len(parts) != 4:
                errors.append(f"Line {line_num}: wrong columns")
                continue
            id_val, x, y, deg = parts
            
            # Check ID format
            try:
                n_str, idx_str = id_val.split('_')
                n = int(n_str)
                idx = int(idx_str)
                expected_id = f"{n:03d}_{idx}"
                if id_val != expected_id:
                    errors.append(f"Line {line_num}: ID '{id_val}' should be '{expected_id}'")
            except:
                errors.append(f"Line {line_num}: Invalid ID format '{id_val}'")
            
            # Check 's' prefix
            if not x.startswith('s'):
                errors.append(f"Line {line_num}: x missing 's' prefix: {x[:20]}")
            if not y.startswith('s'):
                errors.append(f"Line {line_num}: y missing 's' prefix: {y[:20]}")
            if not deg.startswith('s'):
                errors.append(f"Line {line_num}: deg missing 's' prefix: {deg[:20]}")
    
    return errors

print("Validating submission format...")
errors = validate_submission('/home/submission/submission.csv')
if errors:
    print(f"ERRORS FOUND ({len(errors)}):")
    for e in errors[:20]:
        print(f"  {e}")
    print("\n❌ SUBMISSION HAS FORMAT ERRORS - DO NOT SUBMIT!")
else:
    print("✅ All format requirements passed!")

In [ ]:
# Verify the score by re-loading
verify_raw = load_snapshot_with_full_validation('/home/submission/submission.csv')
if verify_raw:
    verify_total = sum(calculate_score_from_rows(verify_raw[n], n) for n in range(1, 201))
    print(f"Verified score: {verify_total:.6f}")
    print(f"Expected score: {new_total:.6f}")
    print(f"Match: {abs(verify_total - new_total) < 1e-10}")
else:
    print("ERROR: Verification failed - format issues in output!")

In [ ]:
# Save metrics
metrics = {
    'cv_score': new_total,
    'baseline_score': baseline_total,
    'improvement': baseline_total - new_total,
    'improvements_found': improvements_found,
    'unique_sources': len(sources),
    'n_values_improved': len(improved_n),
    'snapshots_skipped': snapshots_skipped,
    'format_errors': len(errors) if errors else 0,
    'notes': 'Fixed format ensemble excluding ALL 4 bad snapshots with ID and s-prefix validation.'
}

with open('/home/code/experiments/006_fixed_format_ensemble/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("Metrics saved.")
print(f"CV Score: {new_total:.6f}")